In [10]:
from preprocessing import * 
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score

from scipy.signal import butter, sosfilt, lfilter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
def filter(signal, band, fs):
  sos = butter(10, band, 'bandpass', fs=fs, output='sos')
  filtered = sosfilt(sos, signal)
  return filtered

In [12]:
filename = "data/A01T.gdf"
data = mne.io.read_raw_gdf(filename)


Extracting EDF parameters from d:\Study\GP\EEG-Motor-Imagery-classifier\data\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...


c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\site-packages\mne\io\edf\edf.py:1123: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
c:\Users\Mohamad Abo Bakr\AppData\Local\Programs\Python\Python39\lib\contextlib.py:124: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


In [13]:
dataframe = data.to_data_frame()
dataframe = dataframe.drop(columns="time")


In [14]:
values = dataframe.values.T

filtered_eeg = np.zeros((9, *values.shape))

for i in range(1, 10):
  filtered_eeg[i-1] = filter(values, [4 * i, 4 * i + 4], 250)
  print(4*i, 4*i+4)




4 8
8 12
12 16
16 20
20 24
24 28
28 32
32 36
36 40


In [15]:
filtered_eeg = np.swapaxes(filtered_eeg, 1, 2)
print(filtered_eeg.shape)

(9, 672528, 25)


In [18]:
  # Get the events 
  events = mne.events_from_annotations(data)
  codes = events[1]
  events = events[0]

  # convert annotations to mne codes
  cfilter = np.asarray(['769', '770', '771', '772'])
  lis = np.asarray([codes[i] for i in cfilter])

  # filter for classes
  ev = events[np.in1d(events[:, 2], lis)]

  # extract the samples from events
  x = np.zeros((288, 9, 313, 25))
  y = np.zeros(288)

  values = dataframe.values
  
  for point in range(len(ev)):
    for i in range(9):
      x[point, i] = filtered_eeg[i, ev[point][0]:ev[point][0]+313]
      y[point] = ev[point][2] - lis[0] + 1
      # x[point] = values[ev[point][0]:ev[point][0]+313]
      # y[point] = ev[point][2] - lis[0] + 1
    
  # Create directory for numpy data
  if not os.path.exists("numpy_data"):
    os.mkdir(os.path.join(os.getcwd(), "numpy_data"))

  # get file name without extension or path
  new_name = os.path.splitext(os.path.basename(filename))[0]
  
  # Save data to numpy arrays
  np.save(f"numpy_data2/{new_name}X", x)
  np.save(f"numpy_data2/{new_name}Y", y)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [23]:
x = np.load(f"numpy_data2/{new_name}X.npy")
y = np.load(f"numpy_data2/{new_name}Y.npy")


[[[[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]]

  [[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]]

  [[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]]

  ...

  [[False False False ... False False False]
   [False False False ... False False False]
   [False False False ... False False False]
   ...
   [False Fals

In [7]:
i = 1
x, y = read_file(i)

X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 100, test_size = 0.2)


In [8]:
filter(sig, [8,12], 1000)

NameError: name 'sig' is not defined